<h1 style='text-aling:center;color:Navy'>  Big Data Systems - Fall 2021  </h1>
<h1 style='text-aling:center;color:Navy'>  Assignment 3  </h1>

***

This assignment is focused on Data Analysis and Visualization, as well as utilizing some Query language.

To complete the assignment, you should complete this notebook by filling in the cells provided.

<b>Submission Deadline: This assignment is due Friday, Mar 17 at 8:59 P.M.</b>

A few notes before you start:
- Directly sharing answers is not okay, but discussing problems with other students is encouraged.
- You should start early so that you have time to get help if you're stuck.

- Before continuing the assignment, select "Save and Checkpoint" in the File menu and then execute the submit cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission. If you mistakenly submit the wrong one, you can head to okpy.org and flag the correct version. There will be another submit cell at the end of the assignment when you finish!

<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>
<br>
Before you begin completing the assignment, execute the following cell to load the provided tests.

In [ ]:
# Don't change this cell; just run it. 
# When you log-in please hit return (not shift + return) after typing in your email
from client.api.notebook import Notebook
ok = Notebook('Assignment3.ok')

In [ ]:
_ = ok.submit()

<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>

<hr style="border-top: 5px solid purple; margin-top: 1px; margin-bottom: 1px"></hr>

# <span style="font-size:35px;color:#3665af">Section 1: Data Analysis </span>

This lab will explore the use of Pandas for performing data analysis and MatplotLib for visualizing the results. We will be exploring these tools using a publicly available genome annotation file often used in DNA analysis pipelines. Don't worry if you aren't familiar with genetics, the techniques for managing the data will be the same as any other data set the only difference will be in interpreting the results which is beyond the scope of this assignment.

### Template Data Analysis Pipeline
These are the steps we will be exploring in this lab in relation to a larger data analysis pipeline:
#### 1. Load Data
In this section, we will explore both loading a small file into Pandas directly. In the other section of this assignment we  explore loading data from a remote database using Google BigQuery.

#### 2. Explore the Data
It is very helpful to understand what the data looks like, the types of attributes, number of rows, distribution of values for attributes, etc. Pandas provides functions like ```.head()```, ```.tail()``` and ```.describe()``` to explore the data more analytically. Matplotlib can be used to visualize the data and is very useful for spotting distributions or relations between values. This information can be used to validate our experiment, and inform our predictive model (again, beyond the scope of this assignment).

#### 3. Clean the Data
Data is never perfect, we'll have missing values, extreme outliers, or values that just make no sense. Luckily, Pandas provides functions like ```.isna()``` to identify missing values. Sometimes cleaning the data can be very involved, as in the case of DNA analysis, but this lab focuses on cleaning by removing extraneous data.

***

In [1]:
# import the packages you'll need
import pandas as pd
import matplotlib.pyplot as plt

<div style="font-size:30px;color:#3665af;background-color:#E9E9F5;padding:10px;">1. Loading Data </div>

### Gene Annotation
This section of the assignment will begin by loading a tabular file into a Pandas dataframe, exploring that data, then cleaning it and visualizing the results. This file contains gene annotations, which are regions of DNA that are 'read' in a cell and contain all the data that makes up an organism, like a human. 

The first line of code retrieves the data file from an online repository, which is similar in format to a csv file. It can be read using the `read_csv()` method provided by Pandas with tweeked parameters for the specifics of this file type. 

Since this file is not _exactly_ in a supported file format, the process for loading it has been done for you. But some things to note:
1. Pandas has its own command for reading files by type, which loads directly into a dataframe
2. We can define our own column names that will appear as the header of the dataframe when Pandas loads the file 
3. We can specify how Pandas should read the file i.e separator, comment tokens, in chucks, etc.

In [ ]:
# grab the file from the internet 
!wget ftp://ftp.ensembl.org/pub/release-85/gff3/homo_sapiens/Homo_sapiens.GRCh38.85.gff3.gz
    
# define the column names, which are not provided in the file
col_names = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']

# load the file into a Pandas dataframe
# compression = file is compressed using gzip, specify this to read compressed file directly
# sep = separator, how each separate data value is separated
# comment = lines beginning with this symbol are ignored
# low_memory = process whole file at once rather than chunks, prevents mixed types
# header = file has no header, tell method not to look for one
# names = specify custom column names from list above
annotations = pd.read_csv('Homo_sapiens.GRCh38.85.gff3.gz', compression='gzip',
                         sep='\t', comment='#', low_memory=False,
                         header=None, names=col_names)

<div style="font-size:30px;color:#3665af;background-color:#E9E9F5;padding:10px;">2. Exploring the Data </div>

Start by simply printing out the first 10 rows of the annotations to get a sense for the data and its schema. Pandas has a single method for doing this, which can be found in the [documentation for Pandas Dataframes](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). 

### Question 1
Print out the _first 10_ rows of the annotations dataset

In [ ]:
# INSERT CODE HERE

<div style="font-size:20px;color:#F1F8FC;background-color:#0095EA;padding:10px;">2.1. Closer Look </div>

There are 25 chromosomes in this human genome (22 numbered + X + Y + MT [MT = mitochondrial, if you're wondering]).
chromosomes are physical divisions of the sequence that can be analysed independently in most cases, making analysis easier by focusing on a smaller portion of the whole sequence. The boundaries of each chromosome in the whole sequence are specified in the annotation file, but first we need to check out how the chromosomes are defined in the annotation file. The chromosome is indicated in the column `seqid`

### Question 2
Print out the _unique_ `seqid` values of the annotations dataset

In [ ]:
# INSERT CODE HERE

### Question 3
Summarize the annotations dataset. Provide the count, mean, std, min, 25%, 50%, 75%, max values 

In [ ]:
# INSERT CODE HERE

<div style="font-size:30px;color:#3665af;background-color:#E9E9F5;padding:10px;">3. Cleaning the Data </div>


You'll notice there are a lot more than 25 values. The extra values come from regions that are annotated but not included in the main genome because it is not clear where they belong. We will ignore these by filtering them out.

We can do this by creating a list of value we want to keep (or reject, but that list would be longer), then query the dataframe for rows with values _isin_ (hint) the list.

### Question 4
Filter and print the number of unique `seqid's`

In [ ]:
# list containing seqid values to keep 
chrs = [str(i) for i in range(1,23)] + ['X', 'Y', 'MT']

# query annotations with relevant seqid labels
chrs_only = annotations[ __code_here__ ]

# print the number of unique seqid's to make sure you filtered correctly
len(chrs_only.__code_here__)

<div style="font-size:20px;color:#F1F8FC;background-color:#0095EA;padding:10px;">3.1. Verifying </div>

Now that we have the completed portions (the 25 chromosomes) of the genome separated, lets see how much (percent) of the total genome is unassembled. This will help verify how much of the data we will be missing out on if we only base our analysis on the annotated chromosomes compared to using all annotated data. Luckily, there are rows that sumamrize each region (chromosome and unassembled) and contain information about their entire length.

First, extract only the rows that summarize each `seqid` region (indicated by a `source` equal to GRCh38). Then, find only unassembled regions and create a new Series containing the length of each region. Finally, sum up the lengths of each chromosome we found in the last cell.

### Question 5
Find the length of incomplete regions, find the length of genome sequence, and find the percentage

In [ ]:
# A source of GRCh38 is used for rows that provide a summary of each seqid value
GRCh38_only = annotations[annotations.source == 'GRCh38']

# find the length of the incomplete regions
# (length = end - start)
incomplete_regions = GRCh38_only[GRCh38_only.type == 'supercontig']
incomplete_lens = incomplete_regions.__code_here__

# then find the total length of the genome sequence (hint: each chromosome's length is given in the end column)
total_len = chrs_only[chrs_only.source == 'GRCh38'].__code_here__

# find the percentage
incomplete_ratio = incomplete_lens.sum() / total_len
'{0:.2f}%'.format(incomplete_ratio * 100)

<div style="font-size:20px;color:#F1F8FC;background-color:#0095EA;padding:10px;">3.2. Plotting </div>


Now that we have the chromosomes we want to focus on, we can explore the genes within them. Genes can be any length, but it will be helpful to know the range of length values for choosing an appropriate model in the downstream analysis (beyond the scope of this assignment, but think machine learning or statistical model).

The most helpful plot in this situation will likely be a histogram, which will capture the number of genes in a set range of lengths. Which means first, we will find the length of each gene in the annotation file and store as a new column in a dataframe. Then, use Matplotlib to generate a histogram plot with 50 bins for the length and a log-scale y-axis, use the [Matplotlib `hist` documentation](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) to find the parameters needed.

### Question 6
Get the correct genes, compute and store the length of each gene, and create a plot

In [ ]:
# first, get only genes from annotated regions (type == gene)
genes = chrs_only[ __code_here__ ]

# then complute the length of each gene and store as a new column
# notice the syntax for adding a new column is fairly simple
genes['length'] = __code_here__

# setup the matplotlib hist plot with 50 bins and logarithmic scale on the y-axis
plt.hist(__code_here__)
plt.ylabel('Count')
plt.xlabel('Gene Length (bp)')
plt.show()

<div style="font-size:20px;color:#F1F8FC;background-color:#0095EA;padding:10px;">3.3. More Complex Plot </div>

Now we're going to make a slightly more complex plot to display 3 variables together in a single 2D plot. Specifically we will plot gene count by chromosome and relate it to chromosome length in a scatter plot. 

To do this we will need the lengths of each chromosome, and the number of genes in each chromosome. Finding the lengths will be similar to part 3.1, **but** make sure to keep the `seqid` column to set as the new index to join the two dataframes later.

The genes per chromosome will also require the `seqid` column for joining, and you will need to use the `reset_index()` function before setting `seqid` as the new index.

### Question 7
Get chromosome length, count the genes for each chromosome, join the dataframes, and then plot the count against chromosome length

In [ ]:
# first, get the chromosome lengths (only need entries with type==chromosome)
chr_lens = __code_here__

# next, count genes for each chromosome
genes_per_chr = __code_here__

# join the dataframes on seqid
plot_frame = __code_here__

# then plot the count against the chromosome length
plt.scatter(plot_frame['end'], plot_frame['source'])
plt.ylabel('Gene Length (bp)')
plt.xlabel('Chromosome Length (bp)')
# labels for each point
for idx, p in plot_frame.iterrows():
    plt.annotate(idx, xy=(p[1], p[0]))
plt.show()

<div style="font-size:30px;color:#3665af;background-color:#E9E9F5;padding:10px;">4. Concluding Remarks </div>


In this part we introduced loading, exploring, and cleaning a small dataset using the Pandas and Matplotlib libraries for Python. This data could be used in later analysis for analyzing genes at the chromosome-level, a process known as Variant Calling in a standard genome analysis pipeline. This data could also be used in a machine learning, statistical, or other predictive model to make data-driven decisions.

Both Pandas and Matplotlib have much more depth than what was presented here, but you can leverage a similar analysis pattern to other datasets using the specific tools to transform and visualize as needed.

<div style="font-size:30px;color:#3665af;background-color:#e1dfb1;padding:10px;">5. Exercise </div>

This exercise will have you load, explore, and clean a dataset. The dataset is called train.csv and can be found in Jupyterhub.

- Load and display the first 10 rows
- Summarize the dataset
- Remove columns with any missing values
- Create a 2D scatter plot with the following 3 variables: LotArea, YearBuilt, and SalePrice

In [ ]:
# Load and display the first 10 rows

In [ ]:
# Summarize the dataset

In [ ]:
# Remove columns with any missing values

In [ ]:
# Create a 2D scatter plot with the following 3 variables: LotArea, YearBuilt, and SalePrice

<hr style="border-top: 5px solid purple; margin-top: 1px; margin-bottom: 1px"></hr>

# <span style="font-size:35px;color:#3665af">Section 2: Query Language </span>


In this section of the assignment, we will be using Google BigQuery to query data from the "1000 Genomes Project", a publicly available database that contains information regarding known variants, genetic aberrations that can be the underlying cause of a disease. We will query this information and calculate a single analytical metric, which could normally be used to help verify the results of a DNA analysis pipeline, but in this case the data has been verified already so it can be used to verify your query instead.

Information about the datasets can be found [here](http://googlegenomics.readthedocs.io/en/latest/use_cases/discover_public_data/1000_genomes.html).

In [ ]:
# import a library for connecting Pandas to BigQuery
import pandas as pd
import pandas_gbq as pdgbq

<div style="font-size:30px;color:#3665af;background-color:#E9E9F5;padding:10px;">1. Setup Connection </div>

Connect to BigQuery and run a test query, this will require a project with credits, enabling BigQuery API for your project, and authorization for this notebook to use BigQuery.

To enable the BigQuery API for your project go [here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery).

Running the sample query below will provide a prompt to allow access for this notebook.

The pandas BigQuery extension will return the results of each query into a Pandas Dataframe for further analysis.

In [ ]:
# here is a template query you can use to test your connection: 
query = 'SELECT * '
query += 'FROM `genomics-public-data.1000_genomes.sample_info` '
query += 'LIMIT 25'

# Insert your BigQuery Project ID Here
# Can be found in the Google web console https://console.cloud.google.com
projectid = "__project_id_here__"

# run a simple query, here we print the results without storing them so you can see it is a dataframe
# NOTE!! Google will ask you to authorize pandas when running for the first time!
pdgbq.read_gbq(query, projectid)

<div style="font-size:30px;color:#3665af;background-color:#E9E9F5;padding:10px;">2. Writing a Query </div>


Now we can try a slightly larger query. The 1000 Genomes variant dataset contains over 3TB of genetic variants, which we can run analytical queries on in seconds using BigQuery.

We will begin by querying the dataset to get the reference_name, reference_bases, and alternate_bases that contain only one reference_base *and* one alternate_base. We will test these in the next step.

Query the [genomics-public-data:1000_genomes.variants] table. The results are >40M rows, so you can limit to ~50,000 rows to make the query run in a reasonable amount of time.

BigQuery SQL documentation can be found [here](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators)

### Question 1
Write a query to get the reference_name, reference_bases, and alternate_bases that contains only <b>one</b> reference_base *and* <b>one</b> alternate_base

In [ ]:
query = '__code_here__ '

# this query will take some time depending on how many rows you want
data = pdgbq.read_gbq(query, projectid)

<div style="font-size:30px;color:#3665af;background-color:#E9E9F5;padding:10px;">3. Interpreting the Results </div>


We have provided two functions that can be applied to the results of the above query. The functions calculate whether a variant was a transition or a transversion. We can compute these, then take the ratio of the sum of each as a metric for verifying our dataset (in this case verifying the query).

Remeber that the query returns a dataframe, so you can use any function from the Pandas library that applies to a dataframe. 

#### Below is an explanation of where this metric comes from for those interested, but reading the below paragraph is not required for this assignment!
Transitions/Transversions are terminology for an interesting effect of the chemistry of DNA, which is made of 4 molecules divided into 2 shapes (pyrimidines and purines). A molecule of one shape is more likely to change to the other of the same shape than it is to one of the opposite shape. We can look at the ratio of these transitions to transversions over our entire dataset or a sampling of it as one of a few metrics to verify if the variants we found are correct. Genomes will have a set range of valid Ti/Tv ratios depending on the organism (humans are ~2.1-2.8). [For more about the molecules and their shapes](https://en.wikipedia.org/wiki/Nucleobase).

### Question 2
Apply the transitions and transversions

In [ ]:
pyrimidines = ['A', 'G']
purines = ['T', 'C']

# transitions are a mutation to a base with a similar shape (e.g pyrimidine -> pyrimidine)
def transitions(row):
    if row['reference_bases'] in pyrimidines and row['alternate_bases'] in pyrimidines:
        return 1
    elif row['reference_bases'] in purines and row['alternate_bases'] in purines:
        return 1
    else:
        return 0
    
# transversions are a mutation to a base with a different shape (e.g pyrimidine -> purine)
def transversions(row):
    if row['reference_bases'] in pyrimidines and row['alternate_bases'] in purines:
        return 1
    elif row['reference_bases'] in purines and row['alternate_bases'] in pyrimidines:
        return 1
    else:
        return 0
    
# apply the functions above to the data set 
# queried from the 1000 Genomes database

# ti = apply transitions
data_ti = data.__code_here__
# tv = apply transversions
data_tv = data.__code_here__

# the final output should be between 2.1-2.8, which is normal for humans
print(data_ti.sum() / data_tv.sum())

<div style="font-size:30px;color:#3665af;background-color:#E9E9F5;padding:10px;">4. More Advanced Query </div>


Now we will run a query using a user defined function (UDF) which in this case is small enough to include in the query. We have provided the function (written in Javascript) for you to examine the semantics of if you want to try writing your own in the future. 

This query will require the Google BigQuery client for Python (we are no longer using Pandas) due to the UDF. This will also require a key from your account, instructions for which are provided below.

In [ ]:
# Imports the Google Cloud client library
from google.cloud import bigquery
from google.oauth2 import service_account

You will need to add credentials to run BigQuery from this Jupyter notebook
go to [this page](https://console.cloud.google.com/apis/credentials/serviceaccountkey), then
select this notebook for the account, select JSON and generate the key.
the key will download to your device, then either copy it to the directory
where this notebook is located and paste the name of the JSON file below,
or paste the full path to the JSON file

In [ ]:
# link credentials file
credentials = service_account.Credentials.from_service_account_file('__JSON_path_here__')

# Instantiates a client
bigquery_client = bigquery.Client(project=projectid, credentials=credentials)

### Question 3
Count the number of transversions that occur in each chromosome using the UDF.

Hint: 
This query will again select the reference_name, and reference_bases and alternate_bases of length one. This time though, we will be counting per reference_name using the UDF provided. The trick here is that alternate_bases are an array of strings in this dataset, so you will need to turn that array into a single string somehow before checking its length. The next cell will print out the results of your query and the query will not actually run until the results are printed (lazily evaluated), so if you want to experiment with different queries, you can use the BigQuery interface [here](https://bigquery.cloud.google.com/table/genomics-public-data:1000_genomes.variants).

In [ ]:
# count the number of transversions that occur in each chromosome
# (The  UDF)
query = 'CREATE TEMPORARY FUNCTION tv(ref STRING, alt STRING) '
query += 'RETURNS INT64 '
query += r'''LANGUAGE js AS """ '''
query += 'var pyr = ["A", "G"]; '
query += 'var pur = ["T", "C"]; '
query += 'if ((pyr.includes(ref) && pur.includes(alt)) || (pur.includes(ref) && pyr.includes(alt))) {return 1;} '
query += 'else {return 0;}'
query += r''' """; '''

# select the reference_name and the count of tv's using the above function, note function name and parameters above
query += 'SELECT __code_here__ '
query += 'FROM `genomics-public-data.1000_genomes.variants` '
query += '__code_here__'

data = bigquery_client.query(query)

In [ ]:
# run this to print the results of your query
# notice how this method differs from the Pandas extension that stores the query results in a dataframe
print("Chromosome, Count")
for row in data:
    print(row[0] + ', ' + str(row[1]))

<div style="font-size:30px;color:#3665af;background-color:#E9E9F5;padding:10px;">5. Concluding Remarks </div>


This was a quick look at BigQuery for running analytical queries on a publically available genomics dataset to explore the Ti/Tv ratio. This technique can be applied to DNA sequencing experiments to verify the results of the data cleaning and analysis. If you are interested in the DNA analysis pipeline and BigQuery, an even more advanced version of this query with UDF's can be found [here](https://cloud.google.com/life-sciences/docs/how-tos/analyze-variants).

BigQuery is similar to other big data query languages such as Apache Hive or Impala, so you can apply the information here to the other languages. 

<hr style="border-top: 5px solid purple; margin-top: 1px; margin-bottom: 1px"></hr>

<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>

Submission: Once you're finished, select "Save and Checkpoint" in the File menu and then execute the submit cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission. If you mistakenly submit the wrong one, you can go to the URL that you got at the very beginning of this homework and flag the correct version. To do so, go to the website, click on this assignment, and find the version you would like to have graded. There should be an option to flag that submission for grading. Good luck!

In [ ]:
_ = ok.submit()

<hr style="border-top: 5px solid orange; margin-top: 1px; margin-bottom: 1px"></hr>